### Libraries

In [3]:
'generic imports'
import os
import pandas as pd
import sys
sys.path.append(os.path.abspath('..'))
from src import utils
import numpy as np
import datetime

'machine learning library imports'
from sklearn import tree
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

### Load data

In [4]:
data_dir = os.path.abspath('data')

# Non-augmented dataset
df_train = pd.read_csv(os.path.join(data_dir, 'train_smotenc.csv'), low_memory=False)
AUGMENTATION = 'None'

# SMOTE augmented dataset
# df_train = pd.read_csv(os.path.join(data_dir, 'EdgeIIot_train_100k_SMOTE.csv'), low_memory=False)
# AUGMENTATION = 'SMOTE'

# SMOTE-NC augmented dataset
# df_train = pd.read_csv(os.path.join(data_dir, 'EdgeIIot_train_100k_SMOTE_NC.csv'), low_memory=False)
# AUGMENTATION = 'SMOTE-NC'

# RealTabFormer augmentation dataset
# df_train = pd.read_csv(os.path.join(data_dir, 'EdgeIIot_train_100k_RealTabFormer.csv'), low_memory=False)
# AUGMENTATION = 'RealTabFormer'

# GReaT augmentation dataset
# df_train = pd.read_csv(os.path.join(data_dir, 'EdgeIIot_train_100k_GReaT.csv'), low_memory=False)
# AUGMENTATION = 'GReaT'

# Test data for all datasets
df_test = pd.read_csv(os.path.join(data_dir, 'EdgeIIot_test.csv'), low_memory=False)

FileNotFoundError: [Errno 2] No such file or directory: '/home/tiagociic/Projectos/inovmineral/project/notebooks/data/train_smotenc.csv'

### Data preparation

In [3]:
# Drop columns mbtcp.unit_id and mbtcp.trans_id from train and test data    
#df_train = df_train.drop(['mbtcp.unit_id', 'mbtcp.trans_id'], axis=1)
#df_test = df_test.drop(['mbtcp.unit_id', 'mbtcp.trans_id'], axis=1)

# Creates X_train, y_train
X_train = df_train.drop(['Attack_type'], axis=1)
y_train = df_train['Attack_type']

# Creates X_test, y_test
X_test = df_test.drop(['Attack_label','Attack_type'], axis=1)
y_test = df_test['Attack_type']

#### Convert categorical features to one-hot encoded features

In [4]:
# Extract categorical features
categorical_features = X_train.select_dtypes(include="object").columns

# Concatenate X_train and X_test
X_comb = pd.concat([X_train[categorical_features], X_test[categorical_features]], axis=0)

# Apply one-hot encoding (get_dummies)
X_comb_enc = pd.get_dummies(X_comb, dtype='int8')

# Split back into X_train and X_test
X_train_enc, X_test_enc = train_test_split(
    X_comb_enc, test_size=len(X_test), random_state=42)

# Print the shape of X_train_enc and X_test_enc
print(f'X_train_enc shape: {X_train_enc.shape}, X_test_enc shape: {X_test_enc.shape}')

X_train_enc shape: (1500000, 53), X_test_enc shape: (381934, 53)


In [5]:
# Drop columns categorical_features from X_train and X_test 
X_train = X_train.drop(categorical_features, axis=1)
X_test = X_test.drop(categorical_features, axis=1)

# Concatenate X_train and X_train_enc
X_train = pd.concat([X_train.reset_index(drop=True), X_train_enc.reset_index(drop=True)], axis=1)
X_test = pd.concat([X_test.reset_index(drop=True), X_test_enc.reset_index(drop=True)], axis=1)

# Print the shape of X_train and X_test
print(f'X_train shape: {X_train.shape}, X_test shape: {X_test.shape}')

X_train shape: (1500000, 92), X_test shape: (381934, 92)


#### Label encoding

In [6]:
# instantiate the label encoder
le = LabelEncoder()

# fit and encode the training labels
y_train = le.fit_transform(y_train)

# encode the test labels
y_test = le.transform(y_test)

print('Attack_type and encoded labels:\n')
for i, label in enumerate(le.classes_):
    print(f'{label:23s} {i:d}')

Attack_type and encoded labels:

Backdoor                0
DDoS_HTTP               1
DDoS_ICMP               2
DDoS_TCP                3
DDoS_UDP                4
Fingerprinting          5
MITM                    6
Normal                  7
Password                8
Port_Scanning           9
Ransomware              10
SQL_injection           11
Uploading               12
Vulnerability_scanner   13
XSS                     14


### Model Training

In [7]:
# Shuffle training data
X_train, y_train = shuffle(X_train, y_train, random_state=42)

# Instantiate the decision tree classifier
dt = tree.DecisionTreeClassifier(random_state=42)

# Train the model
dt_clf = dt.fit(X_train, y_train)

### Model Evaluation

In [8]:
predictions = dt_clf.predict(X_test)

In [9]:
accuracy = metrics.accuracy_score(y_test, predictions)
precisionw = metrics.precision_score(y_test, predictions, average='weighted', zero_division=1)
recallw = metrics.recall_score(y_test, predictions, average='weighted')
f1_scorew = metrics.f1_score(y_test, predictions, average='weighted')
precisionm = metrics.precision_score(y_test, predictions, average='macro', zero_division=1)
recallm = metrics.recall_score(y_test, predictions, average='macro')
f1_scorem = metrics.f1_score(y_test, predictions, average='macro')


print("Model Evaluation Metrics")
print("~~~~~~~~~~~~~~~~~~~~~~~~~")
print("Accuracy: {:.2f}".format(accuracy))
print("~~~~~~~~~~~~~~~~~~~~~~~~~")
print("Precision (Weighted): {:.2f}".format(precisionw))
print("Recall (Weighted): {:.2f}".format(recallw))
print("F1(Weighted): {:.2f}".format(f1_scorew))
print("~~~~~~~~~~~~~~~~~~~~~~~~~")
print("Precision (Macro): {:.2f}".format(precisionm))
print("Recall (Macro): {:.2f}".format(recallm))
print("F1(Macro): {:.2f}".format(f1_scorem))
print("~~~~~~~~~~~~~~~~~~~~~~~~~")


Model Evaluation Metrics
~~~~~~~~~~~~~~~~~~~~~~~~~
Accuracy: 0.89
~~~~~~~~~~~~~~~~~~~~~~~~~
Precision (Weighted): 0.93
Recall (Weighted): 0.89
F1(Weighted): 0.90
~~~~~~~~~~~~~~~~~~~~~~~~~
Precision (Macro): 0.69
Recall (Macro): 0.86
F1(Macro): 0.74
~~~~~~~~~~~~~~~~~~~~~~~~~


#### Save Metrics Results 

In [10]:
# create dictionary for results
results = {
    "model": "Decision Tree",
    "augmentations": AUGMENTATION,
    "timestamp": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "accuracy": accuracy,
    "precision (weighted)": precisionw,
    "recall (weighted)": recallw,
    "f1 (weighted)": f1_scorew,
    "precision (macro)": precisionm,
    "recall (macro)": recallm,
    "f1 (macro)": f1_scorem
    }

# save results to csv   
utils.save_results_to_csv([results], 'results/metrics/dt.csv')

#### Confusion Matrix

In [11]:
conf_mat = metrics.confusion_matrix(y_test, predictions)

attack_labels = ['Backdoor', 'DDoS_HTTP', 'DDoS_ICMP', 'DDoS_TCP', 'DDoS_UDP', 
'Fingerprinting', 'MITM', 'Normal', 'Password', 'Port_Scanning', 'Ransomware', 
'SQL_injection', 'Uploading', 'Vulnerability_scanner', 'XSS']

# Create a dataframe from the confusion matrix
conf_mat_df = pd.DataFrame(conf_mat, 
                            index = attack_labels, 
                            columns = attack_labels)
conf_mat_df.index.name = 'Actual'
conf_mat_df.columns.name = 'Predicted'


# Save the confusion matrix
conf_mat_df.to_csv(f"results/conf_matrix/{results['model']}_{results['augmentations']}.csv")
conf_mat_df

Predicted,Backdoor,DDoS_HTTP,DDoS_ICMP,DDoS_TCP,DDoS_UDP,Fingerprinting,MITM,Normal,Password,Port_Scanning,Ransomware,SQL_injection,Uploading,Vulnerability_scanner,XSS
Actual,,,,,,,,,,,,,,,
Backdoor,4472,48,0,0,0,36,0,6,18,53,34,18,84,1,12
DDoS_HTTP,122,6695,0,0,0,357,0,260,399,0,163,662,571,112,287
DDoS_ICMP,0,0,13473,0,0,28,0,0,0,0,0,0,0,0,0
DDoS_TCP,0,0,0,10009,0,0,0,0,0,0,0,0,0,0,0
DDoS_UDP,0,0,0,0,24601,0,0,0,0,0,0,0,0,0,0
Fingerprinting,2,4,1,0,0,106,0,3,2,18,2,4,1,0,3
MITM,0,0,0,0,0,1,75,0,0,0,0,0,0,0,0
Normal,363,4193,0,0,0,1287,0,241572,8223,1,614,7856,4943,938,2786
Password,35,317,0,0,0,117,0,692,7850,0,54,478,334,59,172
